In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv("/kaggle/input/lgdata2020/User_Data.csv")

In [ ]:
dataset

In [ ]:
# input
X= dataset.iloc[:, [2, 3]].values

# output
y = dataset.iloc[:, 4].values


In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()

X = sc_x.fit_transform(X)

print (X)


In [ ]:
from sklearn.model_selection  import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.30)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(xtrain, ytrain)


In [ ]:
y_pred = classifier.predict(xtest)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, y_pred)

print ("Confusion Matrix : \n", cm)


In [ ]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(ytest, y_pred))


# **Logistic Regression Example 2**

The goal of the project is to predict the binary target, whether the patient has heart disease or not.

https://www.justintodata.com/logistic-regression-example-in-python/
https://www.tutorialspoint.com/numpy/numpy_hstack.htm

In [ ]:
# Step 1
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix

#Step 2  Reading  and 

df = pd.read_csv('/kaggle/input/heart-attack-prediction/data.csv', na_values='?')
print(df.head())

# rename y (column) into target
df = df.rename(columns={'num       ': 'target'})

df['target'].value_counts(dropna=False)

df.info()


# Cleaning 
#To keep the cleaning process simple, we’ll remove:

#the columns with many missing values, which are slope, ca, thal.
#the rows with missing values.

df = df.drop(['slope', 'ca', 'thal'], axis=1)

df = df.dropna().copy()

df.info()

#Step #3: Transform the Categorical Variables: Creating Dummy Variables

df['cp'].value_counts(dropna=False)
print(df['cp'].value_counts(dropna=False))
print("*8**********************************")
df['restecg'].value_counts(dropna=False)
print(df['restecg'].value_counts(dropna=False))

df.to_csv("pre_dummy.csv")
# convert into dummy

df = pd.get_dummies(df, columns=['cp', 'restecg'], drop_first=True)
df.to_csv("cleaned.csv")


# seperate numeric and categorical columns

print("# seperate numeric and categorical columns")

numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
cat_cols = list(set(df.columns) - set(numeric_cols) - {'target'})
cat_cols.sort()

print(numeric_cols)
print(cat_cols)

#Step 4: Split Training and Test Datasets


random_seed = 888
df_train, df_test = train_test_split(df, test_size=0.2, random_state=random_seed, stratify=df['target'])

print("Checking the split")
print(df_train.shape)
print(df_test.shape)
print()
print(df_train['target'].value_counts(normalize=True))
print()
print(df_test['target'].value_counts(normalize=True))


#Step 5: Transform the Numerical Variables: Scaling

scaler = StandardScaler()
scaler.fit(df_train[numeric_cols])

def get_features_and_target_arrays(df, numeric_cols, cat_cols, scaler):
    X_numeric_scaled = scaler.transform(df[numeric_cols])
    X_categorical = df[cat_cols].to_numpy()
    X = np.hstack((X_categorical, X_numeric_scaled))
    y = df['target']
    return X, y

X, y = get_features_and_target_arrays(df_train, numeric_cols, cat_cols, scaler)


#Step 6: Fit the Logistic Regression Model

clf = LogisticRegression(penalty='none') # logistic regression with no penalty term in the cost function.

clf.fit(X, y)


In [ ]:
random_seed = 888
df_train, df_test = train_test_split(df, test_size=0.2, random_state=random_seed, stratify=df['target'])

print("Checking the split")
print(df_train.shape)
print(df_test.shape)
print()
print(df_train['target'].value_counts(normalize=True))
print()
print(df_test['target'].value_counts(normalize=True))

In [ ]:
type(X)

The ten features we’ll be using are:

age: age in years
sex: sex (1 = male; 0 = female)
cp: chest pain type
– 1: typical angina
– 2: atypical angina
– 3: non-anginal pain
– 4: asymptomatic
trestbps: resting blood pressure (in mm Hg on admission to the hospital)
chol: serum cholesterol in mg/dl
fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
restecg: resting electrocardiographic results
– 0: normal
– 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
– 2: showing probable or definite left ventricular hypertrophy by Estes’ criteria
thalach: maximum heart rate achieved
exang: exercise-induced angina (1 = yes; 0 = no)
oldpeak: ST depression induced by exercise relative to rest
We can also take a quick look at the data itself by printing out the dataset.

**Step #7: Evaluate the Model**
https://www.justintodata.com/logistic-regression-example-in-python/

In [ ]:
#Before starting, we need to get the scaled test dataset.

X_test, y_test = get_features_and_target_arrays(df_test, numeric_cols, cat_cols, scaler)

plot_roc_curve(clf, X_test, y_test)

plot_precision_recall_curve(clf, X_test, y_test)

test_prob = clf.predict_proba(X_test)[:, 1]
test_pred = clf.predict(X_test)


print('Log loss = {:.5f}'.format(log_loss(y_test, test_prob)))
print('AUC = {:.5f}'.format(roc_auc_score(y_test, test_prob)))
print('Average Precision = {:.5f}'.format(average_precision_score(y_test, test_prob)))
print('\nUsing 0.5 as threshold:')
print('Accuracy = {:.5f}'.format(accuracy_score(y_test, test_pred)))
print('Precision = {:.5f}'.format(precision_score(y_test, test_pred)))
print('Recall = {:.5f}'.format(recall_score(y_test, test_pred)))
print('F1 score = {:.5f}'.format(f1_score(y_test, test_pred)))

print('\nClassification Report')
print(classification_report(y_test, test_pred))


print('Confusion Matrix')
plot_confusion_matrix(clf, X_test, y_test)

**Step #8: Interpret the Results**

In [ ]:
coefficients = np.hstack((clf.intercept_, clf.coef_[0]))
pd.DataFrame(data={'variable': ['intercept'] + cat_cols + numeric_cols, 'coefficient': coefficients})

**Since the numerical variables are scaled by StandardScaler, we need to think of them in terms of standard deviations. Let’s first print out the list of numeric variable and its sample standard deviation.**

In [ ]:
pd.DataFrame(data={'variable': numeric_cols, 'unit': np.sqrt(scaler.var_)})

For example, holding other variables fixed, there is a 41% increase in the odds of having a heart disease for every standard deviation increase in cholesterol (63.470764) since exp(0.345501) = 1.41.

https://www.displayr.com/what-is-a-roc-curve-how-to-interpret-it/
